In [1]:
### Content-based рекомендательная система
##### HR@10  0.766

# Games RSs

In [2]:
# импорты, которые точно понадобятся
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Данные взяты отсюда - http://jmcauley.ucsd.edu/data/amazon/
# http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz
JSON_DATA_PATH = "./reviews_Video_Games_5.json"
N = 10

## Загрузка данных

In [4]:
import json

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        rows.append((uid, iid, review, rating, dt))
        
    return pd.DataFrame(rows, columns=cols)

In [5]:
%%time
df = get_data_frame()

CPU times: user 2.18 s, sys: 148 ms, total: 2.32 s
Wall time: 2.36 s


## Готовим выборки

In [6]:
def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [7]:
training_df, test_df = split_df_by_dt(df)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 5) (46353, 5)
Размер после очистки: (185427, 5) (19174, 5)


 ## Метрика

In [8]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

In [9]:
def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

test_dict = get_test_dict(test_df)

## CBRS

In [10]:
from tqdm import tqdm

class BasicRecommender(object):
    def __init__(self):
        pass
    
    def get_recs(self, uid, top):
        """Строит рекомендации для пользователя uid
        :return: словарь типа {iid: score, ...}
        """
        return {}
    
    def get_batch_recs(self, uids, top):
        """Строит рекомендации для нескольких пользователей uids
        :return: словарь типа {uid: {iid: score, ...}, ...}
        """
        return {uid: self.get_recs(uid, top) for uid in uids}

In [11]:
#########################################################################
def _prepare_iid_data(df, review_ftr_m): 
    iid_to_row = {}
    rows = []
    # не самый оптимальный group by       
    for row_id, iid in enumerate(tqdm(df.iid.unique())):
        iid_to_row[iid] = row_id
        iid_ftr_m = csr_matrix(
            review_ftr_m[np.where(df.iid == iid)[0]].sum(axis=0)
        )
        rows.append(iid_ftr_m)
    iid_ftr_m = normalize(vstack(rows, format='csr'),axis=1)
    return iid_to_row, iid_ftr_m
#########################################################################
def _prepare_uid_data(df, iid_to_row, iid_ftr_m):  
    uid_to_row = {}
    rows = []

    # gr_df - кусок df с данными одного пользователя 
    for gr_id, gr_df in tqdm(df.groupby("uid")):
        uid = gr_df.uid.values[0]

        # поиск объектов и пользовательских рейтингов для них
        iid_rows = []
        iid_rows_dict=dict()
        ratings = []
        for iid, rating in zip(gr_df.iid.values, gr_df.rating.values):
            if iid in iid_to_row:
                iid_rows.append(iid_to_row[iid])
                ratings.append(rating)
                iid_rows_dict[iid]=rating

        # создание профиля пользователя (учитываем только сам факт ревью, без рейтингов)
        if iid_rows:
            ratings = np.array(ratings).reshape(-1, 1)
            uid_ftr_m_plus = csr_matrix(
                iid_ftr_m[iid_rows].multiply(1.0).sum(axis=0)
            )
            uid_to_row[uid] = len(uid_to_row)
            rows.append(uid_ftr_m_plus)

    uid_ftr_m = normalize(vstack(rows, format='csr'))
    return uid_to_row, uid_ftr_m
#########################################################################  


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from scipy.sparse import vstack
import time

class ContentBasedRecommender(BasicRecommender):
    def __init__(self):
        super(BasicRecommender, self).__init__()
    def fit(self,training_df):
        
        self.vect=TfidfVectorizer(stop_words='english',max_features=64*1024)
        
        print("fit on %d records"%(len(training_df)),flush=True)

        start=time.time()
        self.review_ftr_m = self.vect.fit_transform(training_df.review)
        print("TfidfVectorizer.fit_transform(): %0.1f sec"%(time.time()-start),flush=True)
               
        start=time.time()
        self.iid_to_row, self.iid_ftr_m = _prepare_iid_data(training_df, self.review_ftr_m)
        self.row_to_iid = {row_id: iid for iid, row_id in self.iid_to_row.items()}
        print("_prepare_iid_data(): %0.1f sec"%(time.time()-start),flush=True)
                
        start=time.time()
        self.uid_to_row, self.uid_ftr_m = _prepare_uid_data(training_df, self.iid_to_row, self.iid_ftr_m)   
        self.ftr_iid_m = self.iid_ftr_m.T.tocsr()
        print("_prepare_uid_data() %0.1f sec"%(time.time()-start),flush=True)
             
        #Создаем словарик user-item чтобы в дальнейшем не рекомендовать уже купленные игры
        self.used_items=dict()
        for t in training_df.itertuples():
            self.used_items[(t.uid,t.iid)]=True
    
    def get_recs(self, uid, top=N):
        recs = []
        if uid in self.uid_to_row:
            u_row_id = self.uid_to_row[uid]
            u_row = self.uid_ftr_m[u_row_id]

            # самописный cosine similarity
            u_recs = u_row.dot(self.ftr_iid_m)

            for arg_id in np.argsort(-u_recs.data):
                if len(recs)>=top:
                    #если уже набралось N рекомендаций, на выход!
                    break                    
                row_id = u_recs.indices[arg_id]
                iid=self.row_to_iid[row_id]
                if not ((uid,iid) in self.used_items):
                    #пропускаем уже купленные мгры
                    score = u_recs.data[arg_id]
                    recs.append((iid,score))
        return recs
    
    def get_batch_recs(self, uids, top=N):
        rez=dict()
        print("Get recommendations: ",flush=True)
        for uid in tqdm(uids):
            rez[uid]=dict(self.get_recs(uid,top)[:top])
        return rez

## Финальный тест

In [13]:
%%time
recommender=ContentBasedRecommender()
recommender.fit(training_df)
recs_dict=recommender.get_batch_recs(test_df.uid.unique())
value=hit_ratio(recs_dict,test_dict)   
print("HR: ",value)

fit on 185427 records
TfidfVectorizer.fit_transform(): 23.3 sec


100%|██████████| 10098/10098 [00:10<00:00, 942.14it/s]


_prepare_iid_data(): 10.8 sec


100%|██████████| 22215/22215 [00:31<00:00, 703.61it/s]


_prepare_uid_data() 33.9 sec
Get recommendations: 


100%|██████████| 6815/6815 [00:57<00:00, 119.43it/s]

HR:  0.07659574468085106
CPU times: user 2min 3s, sys: 2 s, total: 2min 5s
Wall time: 2min 5s


In [14]:
print("HR: ",value)

HR:  0.07659574468085106
